In [22]:
import pandas as pd
import numpy as np
import time
import torch
import csv
from util_v3 import *
from data_v3 import load_data, get_train_datasets
from model_v3 import *
from torch.utils.data import dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable

# from mscn.data import load_data

import torch
import torch.nn as nn
import torch.nn.functional as F
# from mscn.data import get_train_datasets, load_data, make_dataset

## load and encode train data by calling the function

In [23]:
dataset_name = 'tpcds'

dicts, column_min_max_vals, labels_train, labels_test, max_num_joins, max_num_predicates, train_data, test_data = get_train_datasets(dataset_name)

Loaded queries
Number of training samples: 9336
Number of validation samples: 1038
Created TensorDataset for training data
Created TensorDataset for validation data


In [29]:
num_epochs = 3
batch_size = 1
hid_units = 256
cuda = False

num_materialized_samples = 0

num_classes = 19

In [30]:
table2vec, column2vec, op2vec, join2vec = dicts

num_materialized_samples = 0

# Train model
sample_feats = len(table2vec) + num_materialized_samples
predicate_feats = len(column2vec) + len(op2vec) + 1
join_feats = len(join2vec)

model = SetConv(sample_feats, predicate_feats, join_feats, hid_units, num_classes)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test_data, batch_size=batch_size)

model.train()

SetConv(
  (sample_mlp1): Linear(in_features=26, out_features=256, bias=True)
  (sample_mlp2): Linear(in_features=256, out_features=256, bias=True)
  (predicate_mlp1): Linear(in_features=30, out_features=256, bias=True)
  (predicate_mlp2): Linear(in_features=256, out_features=256, bias=True)
  (join_mlp1): Linear(in_features=56, out_features=256, bias=True)
  (join_mlp2): Linear(in_features=256, out_features=256, bias=True)
  (out_mlp1): Linear(in_features=768, out_features=256, bias=True)
  (out_mlp2): Linear(in_features=256, out_features=19, bias=True)
)

In [34]:
loss_fn = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    accuracy_hist_train = 0

    for batch_idx, data_batch in enumerate(train_data_loader):
        if batch_idx == 1:

            samples, predicates, joins, targets, sample_masks, predicate_masks, join_masks = data_batch

            samples, predicates, joins, targets = Variable(samples), Variable(predicates), Variable(joins), Variable(targets)            
            sample_masks, predicate_masks, join_masks = Variable(sample_masks), Variable(predicate_masks), Variable(join_masks)

            optimizer.zero_grad()
            outputs = model(samples, predicates, joins, sample_masks, predicate_masks, join_masks)

            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()

            is_correct = (torch.argmax(outputs, dim=1) == targets).float()
            accuracy_hist_train += is_correct.sum()
    accuracy_hist_train /= len(train_data_loader)

    print(f'Epoch {epoch} Accuracy ' f'{accuracy_hist_train: .2f}')

targets type: <class 'torch.Tensor'>
targets value: tensor([3])
outputs type: <class 'torch.Tensor'>
outputs value: tensor([[6.5022e-18, 7.3935e-30, 4.4227e-29, 1.2790e-18, 4.3129e-23, 8.4078e-45,
         0.0000e+00, 2.4989e-33, 1.1612e-27, 7.1330e-13, 6.5341e-25, 4.4829e-40,
         3.5374e-31, 1.0000e+00, 6.7538e-24, 4.4483e-29, 2.7273e-32, 2.4045e-27,
         1.6351e-36]], grad_fn=<SoftmaxBackward0>)
Epoch 0 Accuracy  0.00
targets type: <class 'torch.Tensor'>
targets value: tensor([3])
outputs type: <class 'torch.Tensor'>
outputs value: tensor([[1.8239e-15, 1.5370e-26, 2.1045e-26, 8.7586e-17, 3.1137e-21, 5.9398e-41,
         1.3859e-42, 1.8299e-30, 4.7371e-25, 1.3993e-11, 1.3941e-22, 9.7475e-37,
         2.2382e-28, 1.0000e+00, 1.4373e-21, 3.1126e-26, 2.4036e-29, 3.4383e-25,
         1.7459e-33]], grad_fn=<SoftmaxBackward0>)
Epoch 1 Accuracy  0.00
targets type: <class 'torch.Tensor'>
targets value: tensor([7])
outputs type: <class 'torch.Tensor'>
outputs value: tensor([[7.9531e-2

# Get the final training and validation set predictions

In [27]:
def predict(model, data_loader):
    preds = []

    model.eval()
    for data_batch in enumerate(data_loader):

        samples, predicates, joins, targets, sample_masks, predicate_masks, join_masks = data_batch

        samples, predicates, joins, targets = Variable(samples), Variable(predicates), Variable(joins), Variable(targets)            
        sample_masks, predicate_masks, join_masks = Variable(sample_masks), Variable(predicate_masks), Variable(join_masks)

        outputs = model(samples, predicates, joins, sample_masks, predicate_masks, join_masks)

        for i in range(outputs.data.shape[0]):
            preds.append(outputs.data[i])

    return preds

In [21]:
preds_train = predict(model, train_data_loader)
# is_correct = (torch.argmax(preds_train, dim=1) == labels_train).float()
# print(f'Train accuracy: {is_correct.mean():.4f}')


ValueError: not enough values to unpack (expected 7, got 2)